<a href="https://colab.research.google.com/github/ooutaksaluk/MBT-analysis/blob/main/Main_Battle_Tank_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



**Main Battle Tank analysis**



In [1]:
!pip install anvil-uplink opencv-python-headless numpy torch torchvision
!pip install ultralytics
!pip install torch torchvision
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt

  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


Cloning into 'yolov5'...
remote: Enumerating objects: 17274, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 17274 (delta 1), reused 0 (delta 0), pack-reused 17271 (from 3)
Receiving objects: 100% (17274/17274), 16.12 MiB | 20.84 MiB/s, done.
Resolving deltas: 100% (11856/11856), done.
/content/yolov5


In [11]:
from google.colab import drive
import os

# กำหนดเส้นทางของโฟลเดอร์ Dataset ใน Google Drive
dataset_path = "/content/drive/My Drive/dataset_yolo"

# ข้อมูลที่ต้องการใส่ในไฟล์ dataset.yaml
dataset_yaml = """
train: /content/drive/My Drive/dataset_yolo/images/train
val: /content/drive/My Drive/dataset_yolo/images/val

nc: 2  # จำนวนคลาส
names: ["Sight", "Mantlet"]  # ชื่อคลาส
"""

# สร้างไฟล์ dataset.yaml ใน Google Drive
with open(f"{dataset_path}/dataset.yaml", "w") as f:
    f.write(dataset_yaml)

print("✅ ไฟล์ dataset.yaml ถูกสร้างเรียบร้อย!")



✅ ไฟล์ dataset.yaml ถูกสร้างเรียบร้อย!


In [13]:
import os
import cv2

# กำหนดเส้นทางของภาพและ label
images_path = "/content/drive/My Drive/dataset_yolo/images/train"
labels_path = "/content/drive/My Drive/dataset_yolo/labels/train"

# ตรวจสอบว่าไฟล์ภาพอยู่ในเส้นทางหรือไม่
image_file = os.path.join(images_path, "C1.jpg")
if not os.path.exists(image_file):
    print(f"ไม่พบไฟล์ {image_file}")
else:
    print(f"ไฟล์ {image_file} พบในระบบ")

# ลองโหลดภาพ
image = cv2.imread(image_file)

if image is None:
    print(f"ไม่สามารถโหลดภาพ {image_file} ได้!")
else:
    print(f"โหลดภาพ {image_file} สำเร็จ")


ไม่พบไฟล์ /content/drive/My Drive/dataset_yolo/images/train/C1.jpg
ไม่สามารถโหลดภาพ /content/drive/My Drive/dataset_yolo/images/train/C1.jpg ได้!


In [12]:
import cv2
import os
import random
import numpy as np

# กำหนดโฟลเดอร์ที่เก็บภาพและไฟล์ label
images_path = "/content/drive/My Drive/dataset_yolo/images/train"
labels_path = "/content/drive/My Drive/dataset_yolo/labels/train"

# จำนวนที่ต้องการเพิ่ม
num_augmented_images = 500

# ฟังก์ชันสำหรับการเพิ่มข้อมูล (Data Augmentation)
def augment_image(image, label_file):
    # อ่านข้อมูล label
    with open(label_file, 'r') as f:
        labels = f.readlines()

    # การหมุนภาพ
    angle = random.randint(-30, 30)  # หมุนภาพระหว่าง -30 ถึง 30 องศา
    M = cv2.getRotationMatrix2D((image.shape[1] // 2, image.shape[0] // 2), angle, 1)
    rotated_image = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))

    # การพลิกภาพแนวนอน (Horizontal Flip)
    if random.random() > 0.5:
        rotated_image = cv2.flip(rotated_image, 1)

    # การขยาย/ย่อภาพ (Scaling)
    scale_factor = random.uniform(0.8, 1.2)
    scaled_image = cv2.resize(rotated_image, None, fx=scale_factor, fy=scale_factor)

    # ปรับแสงของภาพ
    brightness = random.randint(-50, 50)
    augmented_image = cv2.convertScaleAbs(scaled_image, alpha=1, beta=brightness)

    # อัปเดตข้อมูล label หลังการหมุนหรือพลิกภาพ
    augmented_labels = []
    for label in labels:
        parts = label.split()
        class_id = int(parts[0])  # เลขคลาส
        x_center = float(parts[1])  # ค่าพิกัด x_center
        y_center = float(parts[2])  # ค่าพิกัด y_center
        width = float(parts[3])  # ความกว้าง
        height = float(parts[4])  # ความสูง

        # คำนวณพิกัดใหม่สำหรับ bounding box (ถ้ามีการพลิกหรือหมุนภาพ)
        if angle != 0:
            # โค้ดสำหรับคำนวณการหมุน
            pass  # จะใช้การคำนวณพิกัดใหม่จากมุมการหมุนที่กำหนด

        if scale_factor != 1:
            x_center *= scale_factor
            y_center *= scale_factor
            width *= scale_factor
            height *= scale_factor

        augmented_labels.append(f"{class_id} {x_center} {y_center} {width} {height}\n")

    return augmented_image, augmented_labels

# สร้างภาพใหม่ 500 ภาพสำหรับแต่ละประเภท
for i in range(1, num_augmented_images + 1):
    # สุ่มเลือกไฟล์จากชุดข้อมูล 'Sight' หรือ 'Mantlet'
    # เพิ่มภาพให้กับ Sight
    class_name = "Sight"
    image_file = os.path.join(images_path, f"C{i}.jpg")
    label_file = os.path.join(labels_path, f"C{i}.txt")

    # อ่านภาพ
    image = cv2.imread(image_file)

    # ตรวจสอบว่าอ่านภาพได้หรือไม่
    if image is None:
        print(f"ไม่สามารถโหลดภาพ {image_file} ได้! ข้ามไฟล์นี้ไป.")
        continue  # ข้ามไฟล์นี้ไป

    # เพิ่มข้อมูล (การหมุน, พลิก, ขยาย, ปรับแสง)
    augmented_image, augmented_labels = augment_image(image, label_file)

    # บันทึกภาพใหม่
    augmented_image_file = os.path.join(images_path, f"C{i}_augmented.jpg")
    cv2.imwrite(augmented_image_file, augmented_image)

    # บันทึกไฟล์ label ใหม่
    augmented_label_file = os.path.join(labels_path, f"C{i}_augmented.txt")
    with open(augmented_label_file, 'w') as f:
        f.writelines(augmented_labels)

    print(f"เพิ่มข้อมูลเสร็จแล้วสำหรับ: {augmented_image_file}")

    # เพิ่มภาพให้กับ Mantlet
    class_name = "Mantlet"
    image_file = os.path.join(images_path, f"M{i}.jpg")
    label_file = os.path.join(labels_path, f"M{i}.txt")

    # อ่านภาพ
    image = cv2.imread(image_file)

    # ตรวจสอบว่าอ่านภาพได้หรือไม่
    if image is None:
        print(f"ไม่สามารถโหลดภาพ {image_file} ได้! ข้ามไฟล์นี้ไป.")
        continue  # ข้ามไฟล์นี้ไป

    # เพิ่มข้อมูล (การหมุน, พลิก, ขยาย, ปรับแสง)
    augmented_image, augmented_labels = augment_image(image, label_file)

    # บันทึกภาพใหม่
    augmented_image_file = os.path.join(images_path, f"M{i}_augmented.jpg")
    cv2.imwrite(augmented_image_file, augmented_image)

    # บันทึกไฟล์ label ใหม่
    augmented_label_file = os.path.join(labels_path, f"M{i}_augmented.txt")
    with open(augmented_label_file, 'w') as f:
        f.writelines(augmented_labels)

    print(f"เพิ่มข้อมูลเสร็จแล้วสำหรับ: {augmented_image_file}")


ไม่สามารถโหลดภาพ /content/drive/My Drive/dataset_yolo/images/train/C1.jpg ได้! ข้ามไฟล์นี้ไป.
ไม่สามารถโหลดภาพ /content/drive/My Drive/dataset_yolo/images/train/C2.jpg ได้! ข้ามไฟล์นี้ไป.
ไม่สามารถโหลดภาพ /content/drive/My Drive/dataset_yolo/images/train/C3.jpg ได้! ข้ามไฟล์นี้ไป.
ไม่สามารถโหลดภาพ /content/drive/My Drive/dataset_yolo/images/train/C4.jpg ได้! ข้ามไฟล์นี้ไป.
ไม่สามารถโหลดภาพ /content/drive/My Drive/dataset_yolo/images/train/C5.jpg ได้! ข้ามไฟล์นี้ไป.
ไม่สามารถโหลดภาพ /content/drive/My Drive/dataset_yolo/images/train/C6.jpg ได้! ข้ามไฟล์นี้ไป.
ไม่สามารถโหลดภาพ /content/drive/My Drive/dataset_yolo/images/train/C7.jpg ได้! ข้ามไฟล์นี้ไป.
ไม่สามารถโหลดภาพ /content/drive/My Drive/dataset_yolo/images/train/C8.jpg ได้! ข้ามไฟล์นี้ไป.
ไม่สามารถโหลดภาพ /content/drive/My Drive/dataset_yolo/images/train/C9.jpg ได้! ข้ามไฟล์นี้ไป.
ไม่สามารถโหลดภาพ /content/drive/My Drive/dataset_yolo/images/train/C10.jpg ได้! ข้ามไฟล์นี้ไป.
ไม่สามารถโหลดภาพ /content/drive/My Drive/dataset_yolo/image

In [ ]:
# import os

# # กำหนดเส้นทางของโฟลเดอร์ train ใน Google Drive
# labels_path = "/content/drive/My Drive/dataset_yolo/labels/train"

# # สร้างโฟลเดอร์หากยังไม่มี
# os.makedirs(labels_path, exist_ok=True)

# # ข้อมูลที่ต้องการสร้างในไฟล์ .txt
# objects = []

# # สร้างข้อมูลสำหรับ 'Sight' (C1 ถึง C45)
# for i in range(1, 46):
#     class_id = 0  # Sight
#     file_name = f"C{i}.txt"
#     label = f"{class_id} 0.5 0.5 0.8 0.8\n"
#     objects.append((file_name, label))

# # สร้างข้อมูลสำหรับ 'Mantlet' (M1 ถึง M45)
# for i in range(1, 46):
#     class_id = 1  # Mantlet
#     file_name = f"M{i}.txt"
#     label = f"{class_id} 0.5 0.5 0.8 0.8\n"
#     objects.append((file_name, label))

# # สร้างไฟล์ .txt
# for file_name, label in objects:
#     file_path = os.path.join(labels_path, file_name)
#     with open(file_path, 'w') as f:
#         f.write(label)

# "✅ ไฟล์ .txt สำหรับ 'Sight' และ 'Mantlet' ถูกสร้างเรียบร้อยแล้ว!"


**Training Model**

In [4]:
!python detect.py --weights /content/yolov5/runs/train/exp/weights/best.pt --img 640 --conf 0.4 --source /content/path/to/your/test/images


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
detect: weights=['/content/yolov5/runs/train/exp/weights/best.pt'], source=/content/path/to/your/test/images, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-399-g8cc44963 Python-3.11.11 torch-2.6.0+cu124 CPU

Traceback (most recent call last):
  File "/content/yolov5/detect.py", line 438, in <mod




**Anvil**



In [8]:
import anvil.server
import anvil.media
import torch
import numpy as np
import cv2
from PIL import Image
from torchvision import transforms

# เชื่อมต่อกับ Anvil
anvil.server.connect("server_LX2D76OV3RCR3XCSOMCFYQZL-JGQCERXRR5DWB5QH")

# โหลดโมเดล YOLOv5
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # ใช้โมเดล YOLOv5s ที่ฝึกเสร็จแล้ว

@anvil.server.callable
def process_image(file):
    print("เริ่มประมวลผลรูปภาพ...")

    # โหลดรูปภาพจาก Anvil
    file_bytes = file.get_bytes()
    nparr = np.frombuffer(file_bytes, np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

    if img is None:
        print("ไม่สามารถโหลดรูปภาพได้!")
        return None

    print("โหลดภาพสำเร็จ กำลังประมวลผล...")

    # แปลงภาพเป็น RGB
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # ใช้โมเดล YOLOv5 สำหรับการทำนาย
    results = model(img_rgb)

    # ดึงผลลัพธ์ (bounding boxes, labels, confidence scores)
    pred_boxes = results.xyxy[0].cpu().numpy()  # [x_min, y_min, x_max, y_max, confidence, class_id]
    pred_scores = pred_boxes[:, 4]  # confidence score
    pred_classes = pred_boxes[:, 5].astype(int)  # class_id

    # กำหนด threshold ความมั่นใจของผลลัพธ์ (เช่น 0.5)
    threshold = 0.5
    for i in range(len(pred_scores)):
        if pred_scores[i] >= threshold:
            x_min, y_min, x_max, y_max = map(int, pred_boxes[i][:4])
            class_id = int(pred_classes[i])
            confidence = float(pred_scores[i])

            # วาดกรอบ Bounding Box
            cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
            cv2.putText(img, f"Class {class_id} ({confidence:.2f})",
                        (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX,
                        0.5, (0, 255, 0), 2)

    #  ส่งผลลัพธ์กลับไปยัง Anvil
    _, img_encoded = cv2.imencode('.jpg', img)
    return anvil.BlobMedia("image/jpeg", img_encoded.tobytes())


#  เริ่มเซิร์ฟเวอร์ Anvil
anvil.server.wait_forever()

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-3-18 Python-3.11.11 torch-2.6.0+cu124 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


เริ่มประมวลผลรูปภาพ...
โหลดภาพสำเร็จ กำลังประมวลผล...


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


KeyboardInterrupt: 